In [ ]:
import os

# DIRECTORY INFORMATION
DATASET = "Dogs"
ROOT_DIR = os.path.abspath('/content/mod')
DATA_DIR = os.path.join(ROOT_DIR, 'DATASET/'+DATASET+'/')
OUT_DIR = os.path.join(ROOT_DIR, 'RESULT/'+DATASET+'/')
MODEL_DIR = os.path.join(ROOT_DIR, 'MODEL/')
LOG_DIR = os.path.join(ROOT_DIR, 'LOGS/'+DATASET+'/')

TRAIN_DIR = "/content/train"    # Train dataset path
TEST_DIR = "/content/sky/test_grey"    #Test dataset path 

# DATA INFORMATION
IMAGE_SIZE = 224
BATCH_SIZE = 1

# RANDOM NUMBER GENERATOR INFORMATION
SEED = 128

# TRAINING INFORMATION
USE_PRETRAINED = False
PRETRAINED = "model1_1.ckpt"
NUM_EPOCHS = 1

In [ ]:
import numpy as np
import cv2
import os


class DATA():

    def __init__(self, dirname):
        self.dir_path = os.path.join(DATA_DIR, dirname)
        self.filelist = os.listdir(self.dir_path)
        self.batch_size = BATCH_SIZE
        self.size = len(self.filelist)
        self.data_index = 0

    def read_img(self, filename):
        img = cv2.imread(filename, 3)
        height, width, channels = img.shape
        labimg = cv2.cvtColor(cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE)), cv2.COLOR_BGR2Lab)
        return np.reshape(labimg[:,:,0], (IMAGE_SIZE,IMAGE_SIZE, 1)), labimg[:, :, 1:]

    def generate_batch(self):
        batch = []
        labels = []
        filelist = []
        for i in range(self.batch_size):
            filename = os.path.join(DATA_DIR, self.dir_path, self.filelist[self.data_index])
            filelist.append(self.filelist[self.data_index])
            greyimg, colorimg = self.read_img(filename)
            batch.append(greyimg)
            labels.append(colorimg)
            self.data_index = (self.data_index + 1) % self.size
        batch = np.asarray(batch)/255
        labels = np.asarray(labels)/255
        return batch, labels, filelist

In [ ]:
import os
import numpy as np
import cv2
import tensorflow.compat.v1 as tf
from google.colab.patches import cv2_imshow

tf.disable_v2_behavior()

def deprocess(imgs):
    imgs = imgs * 255
    imgs[imgs > 255] = 255
    imgs[imgs < 0] = 0
    return imgs.astype(np.uint8)


def reconstruct(batchX, predictedY, filelist):
    for i in range(BATCH_SIZE):
        result = np.concatenate((batchX[i], predictedY[i]), axis=2)
        result = cv2.cvtColor(result, cv2.COLOR_Lab2BGR)
        save_path = os.path.join(OUT_DIR, filelist[i][:-4] + "reconstructed.jpg")
        cv2.imwrite(save_path, result)
        cv2_imshow(result)


class MODEL():

    def __init__(self):
        self.inputs = tf.placeholder(shape=[BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, 1], dtype=tf.float32)
        self.labels = tf.placeholder(shape=[BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, 2], dtype=tf.float32)
        self.loss = None
        self.output = None

    def build(self):
        input_data = self.inputs
        low_level_conv1 = Convolution_Layer(shape=[3, 3, 1, 64], stddev=0.1, value=0.1)
        h = low_level_conv1.feed_forward(input_data=input_data, stride=[1, 2, 2, 1])

        low_level_conv2 =Convolution_Layer(shape=[3, 3, 64, 128], stddev=0.1, value=0.1)
        h = low_level_conv2.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        low_level_conv3 =Convolution_Layer(shape=[3, 3, 128, 128], stddev=0.1, value=0.1)
        h = low_level_conv3.feed_forward(input_data=h, stride=[1, 2, 2, 1])

        low_level_conv4 = Convolution_Layer(shape=[3, 3, 128, 256], stddev=0.1, value=0.1)
        h = low_level_conv4.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        low_level_conv5 =Convolution_Layer(shape=[3, 3, 256, 256], stddev=0.1, value=0.1)
        h = low_level_conv5.feed_forward(input_data=h, stride=[1, 2, 2, 1])

        low_level_conv6 = Convolution_Layer(shape=[3, 3, 256, 512], stddev=0.1, value=0.1)
        h = low_level_conv6.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        mid_level_conv1 = Convolution_Layer(shape=[3, 3, 512, 512], stddev=0.1, value=0.1)
        h1 = mid_level_conv1.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        mid_level_conv2 =Convolution_Layer(shape=[3, 3, 512, 256], stddev=0.1, value=0.1)
        h1 = mid_level_conv2.feed_forward(input_data=h1, stride=[1, 1, 1, 1])

        global_level_conv1 =Convolution_Layer(shape=[3, 3, 512, 512], stddev=0.1, value=0.1)
        h2 = global_level_conv1.feed_forward(input_data=h, stride=[1, 2, 2, 1])

        global_level_conv2 =Convolution_Layer(shape=[3, 3, 512, 512], stddev=0.1, value=0.1)
        h2 = global_level_conv2.feed_forward(input_data=h2, stride=[1, 1, 1, 1])

        global_level_conv3 = Convolution_Layer(shape=[3, 3, 512, 512], stddev=0.1, value=0.1)
        h2 = global_level_conv3.feed_forward(input_data=h2, stride=[1, 2, 2, 1])

        global_level_conv4 = Convolution_Layer(shape=[3, 3, 512, 512], stddev=0.1, value=0.1)
        h2 = global_level_conv4.feed_forward(input_data=h2, stride=[1, 1, 1, 1])

        h2_flat = tf.reshape(h2, [BATCH_SIZE, -1])
        dim = h2_flat.get_shape()[1].value
        global_level_FC1 =FullyConnected_Layer(shape=[dim, 1024], stddev=0.04, value=0.1)
        h2 = global_level_FC1.feed_forward(input_data=h2_flat)

        global_level_FC2 = FullyConnected_Layer(shape=[1024, 512], stddev=0.04, value=0.1)
        h2 = global_level_FC2.feed_forward(input_data=h2)

        global_level_FC3 =FullyConnected_Layer(shape=[512, 256], stddev=0.04, value=0.1)
        h2 = global_level_FC3.feed_forward(input_data=h2)

        fusion_layer =Fusion_Layer(shape=[1, 1, 512, 256], stddev=0.1, value=0.1)
        h = fusion_layer.feed_forward(h1, h2, stride=[1, 1, 1, 1])

        colorization_level_conv1 = Convolution_Layer(shape=[3, 3, 256, 128], stddev=0.1, value=0.1)
        h = colorization_level_conv1.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        h = tf.image.resize_images(h, [56, 56], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        colorization_level_conv2 = Convolution_Layer(shape=[3, 3, 128, 64], stddev=0.1, value=0.1)
        h = colorization_level_conv2.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        colorization_level_conv3 = Convolution_Layer(shape=[3, 3, 64, 64], stddev=0.1, value=0.1)
        h = colorization_level_conv3.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        h = tf.image.resize_images(h, [112, 112], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        colorization_level_conv4 = Convolution_Layer(shape=[3, 3, 64, 32], stddev=0.1, value=0.1)
        h = colorization_level_conv4.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        output_layer = Output_Layer(shape=[3, 3, 32, 2], stddev=0.1, value=0.1)
        logits = output_layer.feed_forward(input_data=h, stride=[1, 1, 1, 1])

        self.output = tf.image.resize_images(logits, [224, 224], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        self.loss = tf.reduce_mean(tf.squared_difference(self.labels, self.output))

    def train(self, data):
        optimizer = tf.train.AdamOptimizer(1e-4).minimize(self.loss)
        saver = tf.train.Saver()
        with tf.Session() as session:
            session.run(tf.global_variables_initializer())
            print('All variables Initialized')
            if USE_PRETRAINED:
                saver.restore(session, os.path.join(MODEL_DIR,PRETRAINED))
                print('Pretrained weights loaded')
            for epoch in range(NUM_EPOCHS):
                avg_cost = 0
                for batch in range(int(data.size/BATCH_SIZE)):
                    batchX, batchY, _ = data.generate_batch()
                    feed_dict = {self.inputs: batchX, self.labels: batchY}
                    _, loss_val = session.run([optimizer, self.loss], feed_dict=feed_dict)
                    print("batch:", batch, " loss: ", loss_val)
                    avg_cost += loss_val / int(data.size/BATCH_SIZE)
                print("Epoch:", (epoch + 1), "cost =", "{:.5f}".format(avg_cost))
              #  log.write("Epoch: " + str(epoch + 1) + " Average Cost: " + str(avg_cost) + "\n")

            save_path = saver.save(session, os.path.join(MODEL_DIR, "model" + str(BATCH_SIZE) + "_" + str(NUM_EPOCHS) + ".ckpt"))
            print("Model saved in path: %s" % save_path)
            #log.write("Model saved in path: " + save_path + "\n")

    def test(self, data):
        saver = tf.train.Saver()
        with tf.Session() as session:
            saver.restore(session, os.path.join(MODEL_DIR, "model" + str(BATCH_SIZE) + "_" + str(NUM_EPOCHS) + ".ckpt"))
            avg_cost = 0
            total_batch = int(data.size/BATCH_SIZE)
            for _ in range(total_batch):
                batchX, batchY, filelist = data.generate_batch()
                feed_dict = {self.inputs: batchX, self.labels: batchY}
                predY, loss = session.run([self.output, self.loss], feed_dict=feed_dict)
                reconstruct(deprocess(batchX), deprocess(predY), filelist)
                avg_cost += loss/total_batch
            print("cost =", "{:.3f}".format(avg_cost))
           # log.write("Average Cost: " + str(avg_cost) + "\n")


In [ ]:
import tensorflow as tf

class Layer():

    def __init__(self, shape, stddev, value):
        self.weights = tf.Variable(tf.truncated_normal(shape=shape, stddev=stddev))
        self.biases = tf.Variable(tf.constant(value=value, shape=[shape[-1]]))

    def feed_forward(self, input_data, stride=None):
        raise NotImplementedError


class Convolution_Layer(Layer):

    def __init__(self, shape, stddev, value):
        super(Convolution_Layer, self).__init__(shape, stddev, value)

    def feed_forward(self, input_data, stride):
        conv = tf.nn.conv2d(input_data, self.weights, stride, padding="SAME")
        output_data = tf.nn.tanh(tf.nn.bias_add(conv, self.biases))
        return output_data


class FullyConnected_Layer(Layer):

    def __init__(self, shape, stddev, value):
        super(FullyConnected_Layer, self).__init__(shape, stddev, value)

    def feed_forward(self, input_data, stride=None):
        fullyconnected = tf.matmul(input_data, self.weights)
        output_data = tf.nn.relu(tf.nn.bias_add(fullyconnected, self.biases))
        return output_data


class Fusion_Layer(Convolution_Layer):

    def __init__(self, shape, stddev, value):
        super(Fusion_Layer, self).__init__(shape, stddev, value)

    def feed_forward(self, mid_features, global_features, stride):
        mid_features_shape = mid_features.get_shape().as_list()
        mid_features_reshaped = tf.reshape(mid_features, [BATCH_SIZE, mid_features_shape[1]*mid_features_shape[2], 256])
        fusion_level = []
        for j in range(mid_features_reshaped.shape[0]):
            for i in range(mid_features_reshaped.shape[1]):
                see_mid = mid_features_reshaped[j, i, :]
                see_mid_shape = see_mid.get_shape().as_list()
                see_mid = tf.reshape(see_mid, [1, see_mid_shape[0]])
                global_features_shape = global_features[j, :].get_shape().as_list()
                see_global = tf.reshape(global_features[j, :], [1, global_features_shape[0]])
                fusion = tf.concat([see_mid, see_global], 1)
                fusion_level.append(fusion)
        fusion_level = tf.stack(fusion_level, 1)
        fusion_level = tf.reshape(fusion_level, [BATCH_SIZE, 28, 28, 512])
        return super(Fusion_Layer, self).feed_forward(fusion_level, stride)


class Output_Layer(Layer):

    def __init__(self, shape, stddev, value):
        super(Output_Layer, self).__init__(shape, stddev, value)

    def feed_forward(self, input_data, stride):
        conv = tf.nn.conv2d(input_data, self.weights, stride, padding='SAME')
        output_data = tf.nn.sigmoid(tf.nn.bias_add(conv, self.biases))
        return output_data


In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import datetime

train_data =DATA(TRAIN_DIR)
print("Train Data Loaded")
        # BUILD MODEL
tf.reset_default_graph()
model =MODEL()
print("Model Initialized")
model.build()
print("Model Built")
        # TRAIN MODEL
model.train(train_data)
print("Model Trained")

In [ ]:
# TEST MODEL
test_data =DATA(TEST_DIR)
print("Test Data Loaded")
model.test(test_data)
print("Image Reconstruction Done")